# CMIP6 Regression and Driver Analysis

### Import Packages

In [ ]:
# Load and preprocess
import load_and_preprocess as lap
import numpy as np
import pandas as pd
import xarray as xr
import os

# Statistics
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import spearmanr, pearsonr, kendalltau, variation, gaussian_kde, skew, kurtosis, shapiro
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan

# Regression models
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, ParameterGrid, RandomizedSearchCV, learning_curve
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.inspection import permutation_importance
from xgboost import XGBRegressor

# For plotting
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import cartopy.crs as ccrs
import seaborn as sns

import matplotlib.cm
from matplotlib import rcParams
from matplotlib.lines import Line2D
import matplotlib.colors as colors
from matplotlib.patches import Patch
import matplotlib.patches as mpatches

# For color map
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm

## Functions

### 3. Statitical Tests

#### General Data Info

In [ ]:
def summarize_standardized_data(X_standardized, y, predictor_vars):
    # Summarize predictors
    X_summary = pd.DataFrame(X_standardized, columns=predictor_vars).describe().transpose()
    # Summarize response variable
    y_summary = pd.DataFrame(y, columns=['bgws']).describe().transpose()
    # Combine summaries
    summary = pd.concat([X_summary, y_summary])
    return summary[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]

#### Plot correlations

In [ ]:
def plot_spearman_correlation(ds, predictor_vars, predictant):
    # Number of regions
    n_regions = len(ds.region)
    # Calculate the number of rows and columns for subplots
    n_cols = 3
    n_rows = np.ceil(n_regions / n_cols).astype(int)

    # Initialize the subplot figure
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4))
    axes = axes.flatten()  # Flatten the array for easy iteration

    for index, region in enumerate(ds.region.values):
        # Select the current region data
        df = ds.sel(region=region).to_dataframe().dropna()

        # Concatenate predictor variables and predictant for correlation
        data = df[predictor_vars + [predictant]]

        # Compute the Spearman correlation matrix
        corr_matrix = data.corr(method='spearman')

        # Plot the heatmap
        sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1, cbar=index == 0, ax=axes[index])

        # Set the title with the region name
        region_name = ds.names.sel(region=region).values
        axes[index].set_title(f"Region: {region_name}")

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Remove any unused subplots
    for i in range(n_regions, n_rows * n_cols):
        fig.delaxes(axes[i])

    plt.show()

#### Plot all distributions

In [ ]:
def plot_all_distributions(ds, predictor_vars):
    num_regions = len(ds.region)
    # Set up the matplotlib figure with a certain number of columns
    cols = 3
    rows = (num_regions // cols) + (num_regions % cols > 0)
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 5), constrained_layout=True)
    axes = axes.flatten()  # Flatten to 1D array for easy iteration
    
    for idx, region in enumerate(ds.region.values):
        # Select data for the region
        df = ds.sel(region=region).to_dataframe().reset_index()
        df = df.dropna(subset=predictor_vars + ['bgws'])  # Drop NaN values for relevant columns only
        
        # Extract predictor variables and the target variable
        X = df[predictor_vars]
        y = df['bgws']
        
        # Standardize the predictors
        scaler = StandardScaler()
        X_standardized = scaler.fit_transform(X)
        
        # Create DataFrame from the standardized predictors
        df_standardized = pd.DataFrame(X_standardized, columns=predictor_vars)
        df_standardized['bgws'] = y.values  # Add non-standardized 'bgws'
        
        # Plotting on the respective subplot
        ax = axes[idx]
        # Create a list to store handles for the legend
        handles = []
        for col in df_standardized.columns:
            # Plot each variable and get the handle
            handle = sns.histplot(df_standardized[col], kde=True, ax=ax, label=col)
            handles.append(handle)

        region_name = ds.names.sel(region=region).values
        ax.set_title(f'Region {region_name}')

        # Only add legend to the first subplot
        if idx == 0:
            ax.legend(title='Variable')
    
    # Hide any unused subplots
    for ax in axes[num_regions:]:
        ax.set_visible(False)

    # Display the plot
    plt.show()

#### Scaling

In [ ]:
def custom_scale(data):
    # Find the absolute maximum value in the data
    max_val = np.max(np.abs(data), axis=0)

    # Scale data by dividing by the max value
    scaled_data = data / max_val

    return scaled_data, max_val

In [ ]:
def scale_data(X, method):
    scaler_data = {}

    if method == 'std':
        scaler = StandardScaler()
        X_standardized = scaler.fit_transform(X)
        scaler_data = {'mean': scaler.mean_, 'std': scaler.scale_}

    elif method == 'minmax':
        scaler = MinMaxScaler(feature_range=(-1, 1))
        X_standardized = scaler.fit_transform(X)
        scaler_data = {'min': scaler.data_min_, 'max': scaler.data_max_}
        
    elif method == 'max':
        X_standardized, max_val = custom_scale(X)
        scaler_data = {'max': max_val}

    elif method == 'no_scaling':
        X_standardized = X
        
    else:
        raise ValueError('Scaling method not known')

    return X_standardized, scaler_data

In [ ]:
def custom_scale(data):
    max_val = np.max(np.abs(data), axis=0)
    scaled_data = data / max_val
    return scaled_data, max_val

##### Test scaling

In [ ]:
def plot_data_with_dual_axis(data, scaled_data, ax, skew_unscaled, skew_scaled, kurt_unscaled, kurt_scaled):
    # Plotting unscaled data
    density = gaussian_kde(data)
    xs = np.linspace(min(data), max(data), 200)
    ax.hist(data, bins=30, alpha=0.5, color='blue', density=True)
    ax.plot(xs, density(xs), color='darkblue')

    # Creating secondary axis for scaled data
    ax2 = ax.twinx()
    density_scaled = gaussian_kde(scaled_data)
    xs_scaled = np.linspace(min(scaled_data), max(scaled_data), 200)
    ax2.hist(scaled_data, bins=30, alpha=0.5, color='orange', density=True)
    ax2.plot(xs_scaled, density_scaled(xs_scaled), color='darkorange')

    # Adding scores as text
    textstr = f'skew_unscaled: {skew_unscaled:.2f}\nskew_scaled: {skew_scaled:.2f}\nkurt_unscaled: {kurt_unscaled:.2f}\nkurt_scaled: {kurt_scaled:.2f}'
    ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=9,
            verticalalignment='top', horizontalalignment='right', bbox=dict(facecolor='white', alpha=0.5))

In [ ]:
def scale_and_plot(ds):
    regions = ds.region.values
    variables = [var for var in ds.data_vars]

    for region in [regions[0]]:
        fig, axs = plt.subplots(1, len(variables), figsize=(5 * len(variables), 4))
        fig.suptitle(f'Region: {ds.names.sel(region=region).values}')

        for i, var in enumerate(variables):
            data = ds[var].sel(region=region).values.flatten()
            data_nonan = data[~np.isnan(data)]
            
            skew_unscaled = skew(data_nonan)
            kurt_unscaled = kurtosis(data_nonan)

            # Scale data
            scaled_data, _ = custom_scale(data_nonan)

            skew_scaled = skew(scaled_data)
            kurt_scaled = kurtosis(scaled_data)

            # Plot histograms with dual-axis
            plot_data_with_dual_axis(data_nonan, scaled_data, axs[i], skew_unscaled, skew_scaled, kurt_unscaled, kurt_scaled)
            axs[i].set_title(var)

        # Adjust legend
        fig.legend(['Unscaled', 'Density Unscaled', 'Scaled', 'Density Scaled'], loc='upper right')
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

### 4. Regression Analysis

#### Build Model

##### Linear Regression Model

In [ ]:
def lr_models(ds_change, predictor_vars, predictant, scaling_method, regression_type='ridge', scoring='r2', cv_folds=5, scaling_back=False):
    """
    Train regularized linear regression models (Ridge, Lasso, or ElasticNet) for each region using Grid Search for hyperparameter tuning,
    perform cross-validation, compute permutation importance, and gather performance metrics.

    Parameters:
    - ds_change: xarray datasets
    - predictor_vars: List of predictor variable names
    - predictant: Name of the predictant variable
    - regression_type: Type of regression ('ridge', 'lasso', 'elasticnet')
    - cv_folds: Number of folds for cross-validation
    - scaling_method: Scaling method (std or minmax)
    - scaling_back: Boolean flag to scale back coefficients

    Returns:
    - A dictionary containing trained models, best hyperparameters, coefficients, cross-validation scores,
      permutation importances, and performance metrics for each region.
    """

    # Initialize dictionaries
    regression_models = {}
    best_hyperparams = {}
    best_scores = {}
    regression_coeffs = {}
    cv_scores_r2 = {}
    cv_scores_mse = {}
    cv_scores_r2_test = {}
    cv_scores_mse_test = {}
    cv_scores_r2_train = {}
    cv_scores_mse_train = {}
    train_data = {}
    test_data = {}
    residuals = {}
    performance_metrics_test = {}
    performance_metrics_train = {}
    permutation_importances_test = {}
    permutation_importances_train = {}
    scaled_data = {}

    # Define parameter grid based on regression type
    if regression_type == 'ridge':
        model = Ridge(random_state=42)
        param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
    elif regression_type == 'lasso':
        model = Lasso(random_state=42)
        param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
    elif regression_type == 'elasticnet':
        model = ElasticNet(random_state=42)
        param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio': [0.2, 0.5, 0.8]}
    else:
        raise ValueError("Invalid regression type. Choose 'ridge', 'lasso', or 'elasticnet'.")

    for region in ds_change.region.values:
        # Data preparation
        df = ds_change.sel(region=region).to_dataframe().reset_index()
        df.dropna(inplace=True)
        X = df[predictor_vars]
        y = df[predictant]
        
        # Get region name
        region_name = ds_change.names.sel(region=region).values

        # Scale the data
        X_scaled, scaled_data[f'{region_name}'] = scale_data(X, method=scaling_method)

        # Train/Test split
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

        # Grid search
        grid_search = GridSearchCV(model, param_grid, cv=cv_folds, scoring=scoring, n_jobs=-1) #r2 neg_mean_squared_error
        grid_search.fit(X_train, y_train)

        # Store best model and hyperparameters
        best_model = grid_search.best_estimator_
        regression_models[f'{region_name}'] = best_model
        best_hyperparams[f'{region_name}'] = grid_search.best_params_
        best_scores[f'{region_name}'] = grid_search.best_score_
        regression_coeffs[f'{region_name}'] = best_model.coef_
        
        # Perform cross-validation and store results
        cv_scores_r2[f'{region_name}'] = cross_val_score(best_model, X_scaled, y, cv=cv_folds, scoring='r2')
        cv_scores_mse[f'{region_name}'] = -cross_val_score(best_model, X_scaled, y, cv=cv_folds, scoring='neg_mean_squared_error')
        cv_scores_r2_train[f'{region_name}'] = cross_val_score(best_model, X_train, y_train, cv=cv_folds, scoring='r2')
        cv_scores_mse_train[f'{region_name}'] = -cross_val_score(best_model, X_train, y_train, cv=cv_folds, scoring='neg_mean_squared_error')
        cv_scores_r2_test[f'{region_name}'] = cross_val_score(best_model, X_test, y_test, cv=cv_folds, scoring='r2')
        cv_scores_mse_test[f'{region_name}'] = -cross_val_score(best_model, X_test, y_test, cv=cv_folds, scoring='neg_mean_squared_error')
        
        # Compute and store performance metrics
        y_pred_test = best_model.predict(X_test)
        mse_test = mean_squared_error(y_test, y_pred_test)
        r2_test = r2_score(y_test, y_pred_test)
        performance_metrics_test[f'{region_name}'] = {'MSE': mse_test, 'R2': r2_test}

        y_pred_train = best_model.predict(X_train)
        mse_train = mean_squared_error(y_train, y_pred_train)
        r2_train = r2_score(y_train, y_pred_train)
        performance_metrics_train[f'{region_name}'] = {'MSE': mse_train, 'R2': r2_train}

        # Compute permutation importance
        perm_importance_test = permutation_importance(best_model, X_test, y_test, n_repeats=20, random_state=42)
        permutation_importances_test[f'{region_name}'] = perm_importance_test['importances']

        perm_importance_train = permutation_importance(best_model, X_train, y_train, n_repeats=20, random_state=42)
        permutation_importances_train[f'{region_name}'] = perm_importance_train['importances']

        # Store train and test data
        train_data[f'{region}'] = (X_train, y_train)
        test_data[f'{region}'] = (X_test, y_test)

    # Convert regression coefficients to DataFrame
    coeffs_df = pd.DataFrame.from_dict(regression_coeffs, orient='index', columns=predictor_vars)

    return coeffs_df, regression_models, best_hyperparams, best_scores, train_data, test_data, cv_scores_r2, cv_scores_mse, cv_scores_r2_train, cv_scores_mse_train, cv_scores_r2_test, cv_scores_mse_test, performance_metrics_test, performance_metrics_train, permutation_importances_test, permutation_importances_train, scaled_data

##### XGBR Model

In [ ]:
def train_xgb_models(ds, predictor_vars, predictant, scaling_method, cv_folds=5):
    """
    Train XGBoost models for each region with Grid Search, Cross-Validation, and Train/Test Split.

    Parameters:
    - ds: xarray dataset
    - predictor_vars: List of predictor variable names
    - predictant: Name of the predictant variable
    - scaling_method: Method for scaling features

    Returns:
    - A dictionary containing trained models, best parameters, best scores, CV scores (R2 and MSE), feature importances, and performance metrics for each region.
    """
    # Initialize dictionaries
    xgb_models = {}
    best_params = {}
    best_scores = {}
    feature_importances = {}
    permutation_importances_test = {}
    permutation_importances_train = {}
    performance_metrics_test = {}
    performance_metrics_train = {}
    cv_scores_r2 = {}
    cv_scores_mse = {}
    cv_scores_r2_test = {}
    cv_scores_mse_test = {}
    cv_scores_r2_train = {}
    cv_scores_mse_train = {}
    scaled_data = {}

    # Parameter grid for GridSearchCV
    param_grid = {
    'n_estimators': [100, 200, 400, 800, 1000, 1100, 1200, 1300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [5, 7, 10, 14],
    'min_child_weight': [0.01, 0.1, 1, 10, 12],
    'lambda': [1, 3],
    'alpha': [0.001, 0.01],
    #'gamma': [0.05, 0.1]
    }
    

    for region in ds.region.values:
        # Data preparation
        df = ds.sel(region=region).to_dataframe().reset_index()
        region_name = ds.names.sel(region=region).values
        df.dropna(inplace=True)
        X = df[predictor_vars]
        y = df[predictant]
        
        X_scaled, scaled_data[f'{region_name}'] = scale_data(X, method=scaling_method)

        # Train/Test split
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
        
        # Further split the training set into training and validation sets for hyperparameter tuning
        X_train_tuning, X_val, y_train_tuning, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
        
        # Initialize XGBoost model
        xgb = XGBRegressor(random_state=42, objective='reg:squarederror')

        # GridSearchCV for hyperparameter tuning on training data
        grid_search = GridSearchCV(xgb, param_grid, cv=cv_folds, n_jobs=-1, scoring='r2')
        grid_search.fit(X_train_tuning, y_train_tuning)

        # Store the best model, parameters, and score for the region
        xgb_models[f'{region_name}'] = grid_search.best_estimator_
        best_params[f'{region_name}'] = grid_search.best_params_
        best_scores[f'{region_name}'] = -grid_search.best_score_ 

        # Update feature_importances assignment
        feature_importances[f'{region_name}'] = xgb_models[f'{region_name}'].feature_importances_
        
        #Compute permutation importance
        perm_importance = permutation_importance(
            xgb_models[f'{region_name}'], X_test, y_test, n_repeats=20, random_state=42
        )
        permutation_importances_test[f'{region_name}'] = perm_importance['importances']
        
        perm_importance_train = permutation_importance(
            xgb_models[f'{region_name}'], X_train, y_train, n_repeats=20, random_state=42
        )
        permutation_importances_train[f'{region_name}'] = perm_importance_train['importances']

        # Performance metrics
        y_pred = xgb_models[f'{region_name}'].predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        performance_metrics_test[f'{region_name}'] = {'MSE': mse, 'R2': r2}
        
        y_pred_train = xgb_models[f'{region_name}'].predict(X_val)
        mse_train = mean_squared_error(y_val, y_pred_train)
        r2_train = r2_score(y_val, y_pred_train)
        performance_metrics_train[f'{region_name}'] = {'MSE': mse_train, 'R2': r2_train}

        # Perform cross-validation and store results
        cv_scores_r2[f'{region_name}'] = cross_val_score(xgb_models[f'{region_name}'], X_scaled, y, cv=cv_folds, scoring='r2')
        cv_scores_mse[f'{region_name}'] = -cross_val_score(xgb_models[f'{region_name}'], X_scaled, y, cv=cv_folds, scoring='neg_mean_squared_error')
        cv_scores_r2_train[f'{region_name}'] = cross_val_score(xgb_models[f'{region_name}'], X_train, y_train, cv=cv_folds, scoring='r2')
        cv_scores_mse_train[f'{region_name}'] = -cross_val_score(xgb_models[f'{region_name}'], X_train, y_train, cv=cv_folds, scoring='neg_mean_squared_error')
        cv_scores_r2_test[f'{region_name}'] = cross_val_score(xgb_models[f'{region_name}'], X_test, y_test, cv=cv_folds, scoring='r2')
        cv_scores_mse_test[f'{region_name}'] = -cross_val_score(xgb_models[f'{region_name}'], X_test, y_test, cv=cv_folds, scoring='neg_mean_squared_error')
        
    feature_importances_df = pd.DataFrame.from_dict(feature_importances, orient='index', columns=predictor_vars)

    return feature_importances_df, permutation_importances_test, permutation_importances_train, best_params, xgb_models, best_scores, performance_metrics_test, performance_metrics_train, cv_scores_r2, cv_scores_mse, cv_scores_r2_train, cv_scores_mse_train, cv_scores_r2_test, cv_scores_mse_test, scaled_data

#### Test Model

##### Test Overfitting

In [ ]:
def plot_learning_curves_for_all_regions(models, ds, predictor_vars, predictant, cv_folds=5):
    """
    Plots learning curves for each region's model using the dataset.

    Parameters:
    - models: Dictionary of trained models, one for each region.
    - ds: xarray dataset used in training models.
    - predictor_vars: List of predictor variable names.
    - predictant: Name of the predictant variable.
    - region_names: List of region names.
    - cv_folds: Number of folds for cross-validation.
    """
    for region in ds.region.values:
        # Extract data for the region
        df = ds.sel(region=region).to_dataframe().reset_index()
        df.dropna(inplace=True)
        X = df[predictor_vars]
        y = df[predictant]
        
        region_name = ds.names.sel(region=region).values

        # Learning curve computation
        train_sizes, train_scores, validation_scores = learning_curve(
            estimator=models[f'{region_name}'],
            X=X, 
            y=y, 
            train_sizes=np.linspace(0.1, 1.0, 10, 550),
            cv=cv_folds,
            scoring='r2',
            n_jobs=-1
        )

        # Calculate mean and standard deviation for training and validation set scores
        train_mean = np.mean(train_scores, axis=1)
        train_std = np.std(train_scores, axis=1)
        validation_mean = np.mean(validation_scores, axis=1)
        validation_std = np.std(validation_scores, axis=1)

        # Plot learning curves
        plt.figure(figsize=(10, 6))
        plt.plot(train_sizes, train_mean, label='Training error', color='blue', marker='o')
        plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.15)
        plt.plot(train_sizes, validation_mean, label='Validation error', color='green', marker='o')
        plt.fill_between(train_sizes, validation_mean - validation_std, validation_mean + validation_std, color='green', alpha=0.15)

        plt.title(f'Learning Curves for {region_name}')
        plt.xlabel('Training Data Size')
        plt.ylabel('R^2')
        plt.legend(loc='upper right')
        plt.grid()
        plt.show()

##### Test Model Performances

In [ ]:
def plot_performance_metrics(performance_metrics_train, performance_metrics_test, cv_scores_r2, cv_scores_mse, cv_scores_r2_train, cv_scores_mse_train, cv_scores_r2_test, cv_scores_mse_test):
    regions = list(performance_metrics_train.keys())
    mse_train = [performance_metrics_train[region]['MSE'] for region in regions]
    mse_test = [performance_metrics_test[region]['MSE'] for region in regions]
    r2_train = [performance_metrics_train[region]['R2'] for region in regions]
    r2_test = [performance_metrics_test[region]['R2'] for region in regions]
   
    fig, ax = plt.subplots(6, 1, figsize=(15, 30))

    # Plot 1: R2 Comparison
    ax[0].plot(regions, r2_train, label='Train R2', marker='o')
    ax[0].plot(regions, r2_test, label='Test R2', marker='o')
    ax[0].set_title('R2 Comparison')
    ax[0].set_xlabel('Regions')
    ax[0].set_ylabel('R2')
    ax[0].set_ylim([0,1])
    ax[0].legend()
    ax[0].tick_params(axis='x', rotation=90)

    # Plot 2: MSE Comparison
    ax[1].plot(regions, mse_train, label='Train MSE', marker='o')
    ax[1].plot(regions, mse_test, label='Test MSE', marker='o')
    ax[1].set_title('MSE Comparison')
    ax[1].set_xlabel('Regions')
    ax[1].set_ylabel('MSE')
    ax[1].set_ylim([0,0.006])
    ax[1].legend()
    ax[1].tick_params(axis='x', rotation=90)

    # Function to plot mean or standard deviation
    def plot_cv_scores(ax, cv_scores_whole, cv_scores_train, cv_scores_test, title, ylabel):
        means_whole = [np.mean(scores) for scores in cv_scores_whole]
        means_train = [np.mean(scores) for scores in cv_scores_train]
        means_test = [np.mean(scores) for scores in cv_scores_test]
        if ylabel == 'Mean Scores':
            ax.plot(regions, means_whole, label='Whole Data', marker='o')
            ax.plot(regions, means_train, label='Train Data', marker='o')
            ax.plot(regions, means_test, label='Test Data', marker='o')
        else:
            stds_whole = [np.std(scores) for scores in cv_scores_whole]
            stds_train = [np.std(scores) for scores in cv_scores_train]
            stds_test = [np.std(scores) for scores in cv_scores_test]
            ax.plot(regions, stds_whole, label='Whole Data', marker='o')
            ax.plot(regions, stds_train, label='Train Data', marker='o')
            ax.plot(regions, stds_test, label='Test Data', marker='o')
        
        # Set y-axis limits for R2 plots
        if title == 'Mean CV R2 Scores' or title == 'Std CV R2 Scores':
            ax.set_ylim([0, 1]) 
        else:
            ax.set_ylim([0, 0.006]) 

        ax.set_title(title)
        ax.set_xlabel('Regions')
        ax.set_ylabel(ylabel)
        ax.legend()
        ax.tick_params(axis='x', rotation=90)

    # Plot 3: Mean CV R2 Scores
    plot_cv_scores(ax[2], [cv_scores_r2[region] for region in regions], [cv_scores_r2_train[region] for region in regions], [cv_scores_r2_test[region] for region in regions], 'Mean CV R2 Scores', 'Mean Scores')

    # Plot 4: Std CV R2 Scores
    plot_cv_scores(ax[3], [cv_scores_r2[region] for region in regions], [cv_scores_r2_train[region] for region in regions], [cv_scores_r2_test[region] for region in regions], 'Std CV R2 Scores', 'Standard Deviation')

    # Plot 5: Mean CV MSE Scores
    plot_cv_scores(ax[4], [cv_scores_mse[region] for region in regions], [cv_scores_mse_train[region] for region in regions], [cv_scores_mse_test[region] for region in regions], 'Mean CV MSE Scores', 'Mean Scores')

    # Plot 6: Std CV MSE Scores
    plot_cv_scores(ax[5], [cv_scores_mse[region] for region in regions], [cv_scores_mse_train[region] for region in regions], [cv_scores_mse_test[region] for region in regions], 'Std CV MSE Scores', 'Standard Deviation')

    plt.tight_layout()
    plt.show()

##### Test model diagnostics

In [ ]:
def test_regression_assumptions_scikit(regression_models, test_data, predictor_vars):
    results = []

    for region_name in regression_models:
        model = regression_models[region_name]
        X_test, y_test = test_data[region_name]

        # Predict and calculate residuals
        predictions = model.predict(X_test)
        residuals = y_test - predictions

        # Add a constant term for the Breusch-Pagan test
        X_test_with_constant = np.column_stack((np.ones(X_test.shape[0]), X_test))

        # Test for normality of residuals
        shapiro_stat, shapiro_p = shapiro(residuals)

        # Test for homoscedasticity
        _, _, _, bp_pvalue = het_breuschpagan(residuals, X_test_with_constant)

        # VIF for multicollinearity
        vif = [variance_inflation_factor(X_test, i) for i in range(X_test.shape[1])]

        # Prepare plot data
        plot_data = {
            'Region': region_name,
            'Predictions': predictions,
            'Residuals': residuals,
            'Shapiro-Wilk': shapiro_stat,
            'Shapiro-Wilk p-value': shapiro_p,
            'Breusch-Pagan p-value': bp_pvalue,
            'VIF': vif
        }
        results.append(plot_data)
    
    # Plotting
    num_regions = len(results)
    fig, axs = plt.subplots(num_regions, 3, figsize=(22, 5 * num_regions)) # Changed to 3 subplots for VIF

    for i, result in enumerate(results):
        sns.residplot(x=result['Predictions'], y=result['Residuals'], lowess=True, ax=axs[i, 0])
        axs[i, 0].set_title(f'Residuals vs Predictions for {result["Region"]}')
        axs[i, 0].set_xlabel('Predicted values')
        axs[i, 0].set_ylabel('Residuals')

        # Adding text for statistical tests
        axs[i, 0].text(0.05, 0.95, f"Shapiro-Wilk: {result['Shapiro-Wilk']:.2f}", transform=axs[i, 0].transAxes)
        axs[i, 0].text(0.05, 0.90, f"Shapiro-Wilk p-value: {result['Shapiro-Wilk p-value']:.2f}", transform=axs[i, 0].transAxes)
        axs[i, 0].text(0.05, 0.85, f"Breusch-Pagan p-value: {result['Breusch-Pagan p-value']:.2f}", transform=axs[i, 0].transAxes)

        sns.histplot(result['Residuals'], kde=True, ax=axs[i, 1])
        axs[i, 1].set_title(f'Residual Distribution for {result["Region"]}')
        axs[i, 1].set_xlabel('Residuals')
        axs[i, 1].set_ylabel('Frequency')

        # VIF bar plot
        sns.barplot(x=predictor_vars, y=result['VIF'], ax=axs[i, 2])
        axs[i, 2].hlines(5, xmin=-0.5, xmax=len(result['VIF'])-0.5, colors='orange', linestyles='dashed')
        axs[i, 2].hlines(10, xmin=-0.5, xmax=len(result['VIF'])-0.5, colors='r', linestyles='dashed')
        axs[i, 2].set_title(f'VIF for {result["Region"]}')
        axs[i, 2].set_xlabel('Predictor Variables')
        axs[i, 2].set_ylabel('VIF Value')
        axs[i, 2].set_ylim(0, max(result['VIF']) + 1)

    plt.tight_layout()
    plt.show()

    return pd.DataFrame(results)

#Linearity: Random pattern without any discernible pattern --> non-linearity
#
#Independence of Errors: Durbin-Watson values between 1.5 and 2.5 are relatively normal
#
#Homoscedasticity (Equal Variance of Errors):  Variance of the residuals is consistent across all levels of the predicted values +
#                                              Breusch-Pagan test: A small p-value (typically <= 0.05)  suggests heteroscedasticity, which can invalidate                                                   some of the statistical conclusions of the regression.
#                                              
#Normality of Errors: Shapiro-Wilk test: The closer this value is to 1, the more the residuals follow a normal distribution. 
#                     Sapiro-Wilk p-value: The p-value from the Shapiro-Wilk test. A small p-value (typically <= 0.05) indicates that                        the residuals do not follow a normal distribution.
#                     
#Variance Inflation Factor (VIF): Measures multicollinearity among the independent variables in the regression model. A VIF value                                        greater than 10 is typically considered an indicator of serious multicollinearity that could affect                                    the model's estimates.    

### 5. Driver Analysis

#### Plot Permutation Importances

In [ ]:
def plot_permutation_importances(permutation_importances, predictor_vars, performance_metrics, split='test', season='Year', save_fig=False):
    # Set larger font sizes
    plt.rcParams.update({'font.size': 24})
    
    # Get variable names and prepare them for display
    var_names = prepare_display_variables(predictor_vars)

    for region in permutation_importances:
        # Convert arrays to DataFrame for easy plotting, applying new variable names for display
        df = pd.DataFrame(permutation_importances[region].T, columns=predictor_vars)
        df.rename(columns=var_names, inplace=True)
        
        # Create the plot for the region
        fig, ax = plt.subplots(figsize=(12, 8))
        sns.boxplot(data=df, orient='h', ax=ax)
        ax.axvline(0, color='grey', linestyle='--')
        
        # Title with region
        #ax.set_title(f'{region} - Permutation Importance ({season})')
        
        # Add Mean Squared Error and R-squared as text within the plot
        mse = round(performance_metrics[region]['MSE'], 4)
        r2 = round(performance_metrics[region]['R2'], 2)
        ax.text(0.95, 0.01, f'Mean Squared Error: {mse}\nR$^2$: {r2}',
                verticalalignment='bottom', horizontalalignment='right',
                transform=ax.transAxes,  # Position text relative to axes
                color='black', fontsize=26)
        
        ax.set_xlabel('Decrease in Accuracy Score')

        plt.tight_layout()
        plt.show()
        
        if save_fig:
            save_figure(fig, region, split, season)

In [ ]:
def prepare_display_variables(variables):
    var_map = {
        'tas': ('T', '°C'),
        'vpd': ('VPD', 'hPa'),
        'gpp': ('GPP', r'\frac{\frac{gC}{m^2}}{day}'),  
        'pr': ('P', r'\frac{mm}{day}'),
        'mrro': ('R', r'\frac{mm}{day}'),
        'evspsbl': ('ET', r'\frac{mm}{day}'),
        'tran': ('Tran', r'\frac{mm}{day}'),
        'evapo': ('E', r'\frac{mm}{day}'),
        'lai': ('Lai', r'\frac{m^2}{m^2}'),
        'mrso': ('SM', '\%'),
        'rgtr': ('P/T', r'\frac{GPP}{T}'),
        'et_partitioning': ('EP', r'\frac{E-Tran}{ET}'),
        'growing_season_length_period': ('GSL', 'days'),
        'RX5day': ('RX5d', 'mm'),
        'growing_season_length_winter': ('GSL', 'days'),
        'growing_season_length_summer': ('GSL', 'days'),
        'growing_season_length_fall': ('GSL', 'days'),
        'growing_season_length_spring': ('GSL', 'days'),
        'wue': ('WUE', r'\frac{GPP}{Tran}'),
        'bgws': ('BGWS', r'\frac{R-Tran}{P}')
    }
    display_variables = {}
    for var in variables:
        if var in var_map:
            abbreviation, units = var_map[var]
            # Enclose units in \left[ and \right] for automatic sizing
            display_variables[var] = f"${{\Delta\, \mathrm{{\it{{{abbreviation}}}}}}}$"
        else:
            print(f"Variable '{var}' not found in var_map.")
            display_variables[var] = var  # Or handle this case as appropriate
    return display_variables

In [ ]:
def save_figure(fig, region, split, season):
    # Caption and figure saving
    region_name = region.replace("/", "_")
    savepath = os.path.join('..', '..', 'results', 'CMIP6', 'historical-ssp370', 'time', 'mean', 'permutation_importance', season, split)
    os.makedirs(savepath, exist_ok=True)
    filename = f'{region_name}_permutation_importance_{split}_data_{season}.pdf'
    filepath = os.path.join(savepath, filename)
    fig.savefig(filepath, dpi=600, bbox_inches='tight', format='pdf')
    print(f'Figure saved under {filepath}')

## 1. Load and Preprocess

In [ ]:
season='winter' # None, 'spring', summer, fall, winter
ds_dict = lap.load_and_preprocess(vars='all', scenarios=['historical', 'ssp370'], models='all', period=season, yearly_sum=False, period_statistic='mean')

## 2. Subdivide Regions, Compute Mean and Change

In [ ]:
ds_dict_regions = lap.subdivide_region_and_compute_mean(ds_dict, with_global=True, spatial_mean=False)

In [ ]:
ds_dict_regions_change = lap.compute_change_dict(ds_dict_regions)

In [ ]:
ds_dict_regions_change['historical-ssp370'] = lap.compute_ensemble(ds_dict_regions_change['historical-ssp370'], 'mean')
ds_dict_regions_change['historical-ssp370'] = lap.compute_ensemble(ds_dict_regions_change['historical-ssp370'], 'median')

## 3. Statistical Tests

#### General Data Info

In [ ]:
summarize_standardized_data(X, y, predictor_vars)

#### Plot correlations

In [ ]:
plot_spearman_correlation(ds, predictor_vars, predictant)

#### Plot all distributions

In [ ]:
plot_all_distributions(ds, predictor_vars)

#### Scale Data and test effects

In [ ]:
cv_df = scale_and_plot(ds)

## 4. Regression Analysis

### Build Model

#### Linear Regression Model

In [ ]:
# Define data and predictor variables
if season==None:
    season='period'
    
predictor_vars = ['pr', 'vpd', 'mrso', 'lai', 'wue', 'RX5day', f'growing_season_length_{season}']  
predictant = 'bgws'

# Run model
(
    coeffs_df_lr, 
    linear_regression_models, 
    best_hyperparams_lr, 
    best_scores_lr, 
    train_data_lr, test_data_lr, 
    cv_scores_r2_lr, cv_scores_mse_lr, 
    cv_scores_r2_train_lr, cv_scores_mse_train_lr, 
    cv_scores_r2_test_lr, cv_scores_mse_test_lr, 
    performance_metrics_test_lr, performance_metrics_train_lr, 
    permutation_importances_test_lr, permutation_importances_train_lr, 
    scaled_data_lr 
)= lr_models(ds_dict_regions_change['historical-ssp370']['Ensemble mean'], 
                           predictor_vars, predictant, scaling_method='max', 
                           regression_type='elasticnet', scoring='neg_mean_squared_error', 
                           cv_folds=4, scaling_back=False)

#### XGBR Model

In [ ]:
# Define data and predictor variables
predictor_vars = ['tas', 'pr', 'vpd','evapo', 'mrso','lai', 'gpp'] #'pr',  'gpp' 'evspsbl'  'mrro', 'tran', 
predictant = 'bgws'
ds_reduced_regions = ds.sel(region=slice(18, 22))
feature_importances_df_xgb, permutation_importances_test_xgb, permutation_importances_train_xgb, best_params_xgb, x_gradient_boosting_models, best_scores_xgb, performance_metrics_test_xgb, performance_metrics_train_xgb, cv_scores_r2_xgb, cv_scores_mse_xgb, cv_scores_r2_train_xgb, cv_scores_mse_train_xgb, cv_scores_r2_test_xgb, cv_scores_mse_test_xgb, scaler_data_xgb = train_xgb_models(ds_reduced_regions, predictor_vars, predictant, scaling_method='max', cv_folds=5)

### Test Model

#### Test Overfitting

In [ ]:
plot_learning_curves_for_all_regions(linear_regression_models, ds_dict_regions_change['historical-ssp370']['Ensemble mean'], predictor_vars, predictant)

#### Plot Performance Metrics

In [ ]:
plot_performance_metrics(performance_metrics_train_lr, 
                         performance_metrics_test_lr, 
                         cv_scores_r2_lr, 
                         cv_scores_mse_lr, 
                         cv_scores_r2_train_lr, 
                         cv_scores_mse_train_lr,
                         cv_scores_r2_test_lr, 
                         cv_scores_mse_test_lr)

#### Test Model Diagnostics

In [ ]:
assumptions_df = test_regression_assumptions_scikit(regression_models, test_data, predictor_vars)

## 5. Driver Analysis

#### Plot Permutation Importances

In [ ]:
if season == 'period':
    season='Year'
    
plot_permutation_importances(permutation_importances_train_lr, predictor_vars, performance_metrics_train_lr, split='train', season=season, save_fig=True)
plot_permutation_importances(permutation_importances_test_lr, predictor_vars, performance_metrics_test_lr, split='test', season=season, save_fig=True)

## Supplements

### Extreme Gradient Boosting Regression Analysis

#### Assess variable importance

In [ ]:
permutation_importance_df_gb

In [ ]:
feature_importances_df * 100

In [ ]:
feature_importances_df * 100

#### Model diagnostics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan

def test_regression_assumptions_scikit(regression_models, test_data, predictor_vars):
    results = []

    for region_name in regression_models:
        model = regression_models[region_name]
        X_test, y_test = test_data[region_name]

        # Predict and calculate residuals
        predictions = model.predict(X_test)
        residuals = y_test - predictions

        # Add a constant term for the Breusch-Pagan test
        X_test_with_constant = np.column_stack((np.ones(X_test.shape[0]), X_test))

        # Test for normality of residuals
        shapiro_stat, shapiro_p = shapiro(residuals)

        # Test for homoscedasticity
        _, _, _, bp_pvalue = het_breuschpagan(residuals, X_test_with_constant)

        # VIF for multicollinearity
        vif = [variance_inflation_factor(X_test, i) for i in range(X_test.shape[1])]

        # Prepare plot data
        plot_data = {
            'Region': region_name,
            'Predictions': predictions,
            'Residuals': residuals,
            'Shapiro-Wilk': shapiro_stat,
            'Shapiro-Wilk p-value': shapiro_p,
            'Breusch-Pagan p-value': bp_pvalue,
            'VIF': vif
        }
        results.append(plot_data)
    
    # Plotting
    num_regions = len(results)
    fig, axs = plt.subplots(num_regions, 3, figsize=(22, 5 * num_regions)) # Changed to 3 subplots for VIF

    for i, result in enumerate(results):
        sns.residplot(x=result['Predictions'], y=result['Residuals'], lowess=True, ax=axs[i, 0])
        axs[i, 0].set_title(f'Residuals vs Predictions for {result["Region"]}')
        axs[i, 0].set_xlabel('Predicted values')
        axs[i, 0].set_ylabel('Residuals')

        # Adding text for statistical tests
        axs[i, 0].text(0.05, 0.95, f"Shapiro-Wilk: {result['Shapiro-Wilk']:.2f}", transform=axs[i, 0].transAxes)
        axs[i, 0].text(0.05, 0.90, f"Shapiro-Wilk p-value: {result['Shapiro-Wilk p-value']:.2f}", transform=axs[i, 0].transAxes)
        axs[i, 0].text(0.05, 0.85, f"Breusch-Pagan p-value: {result['Breusch-Pagan p-value']:.2f}", transform=axs[i, 0].transAxes)

        sns.histplot(result['Residuals'], kde=True, ax=axs[i, 1])
        axs[i, 1].set_title(f'Residual Distribution for {result["Region"]}')
        axs[i, 1].set_xlabel('Residuals')
        axs[i, 1].set_ylabel('Frequency')

        # VIF bar plot
        sns.barplot(x=predictor_vars, y=result['VIF'], ax=axs[i, 2])
        axs[i, 2].hlines(5, xmin=-0.5, xmax=len(result['VIF'])-0.5, colors='orange', linestyles='dashed')
        axs[i, 2].hlines(10, xmin=-0.5, xmax=len(result['VIF'])-0.5, colors='r', linestyles='dashed')
        axs[i, 2].set_title(f'VIF for {result["Region"]}')
        axs[i, 2].set_xlabel('Predictor Variables')
        axs[i, 2].set_ylabel('VIF Value')
        axs[i, 2].set_ylim(0, max(result['VIF']) + 1)

    plt.tight_layout()
    plt.show()

    return pd.DataFrame(results)

In [ ]:
assumptions_df = test_regression_assumptions_scikit(regression_models, test_data, predictor_vars)

#### Test model performance

In [ ]:
def assess_model_performance_all_regions(ds, regression_models, best_params, best_scores, scalers, predictor_vars):
    region_names = ds.names.values
    region_indices = ds.region.values
    
    # Determine the number of rows and columns for the subplots based on the number of regions
    num_regions = len(region_names)
    cols = 3  # We are keeping 3 columns as per your requirement
    rows = math.ceil(num_regions / cols)
    
    fig, axs = plt.subplots(rows, cols, figsize=(15, 5 * rows), squeeze=False)
    
    # Initialize an empty dictionary to store performance metrics for each region
    performance_metrics = {}

    for idx, (ax, region) in enumerate(zip(axs.flatten(), region_indices)):
        region_name = ds.names.sel(region=region).values.item()
        # Prepare the data for the region
        df = ds.sel(region=region).to_dataframe().dropna()
        X = df[predictor_vars]
        y_true = df['bgws'].values

        # Retrieve the scaler for the region
        scaler = scalers[region_name]
        X_standardized = scaler.transform(X)  # Use transform here, not fit_transform
        
        # Predict using the trained model
        model = regression_models[region_name]
        y_pred = model.predict(X_standardized)
        
        # Calculate residuals
        residuals = y_true - y_pred
        
        # Calculate and store performance metrics
        mse_value = mean_squared_error(y_true, y_pred)
        r2_value = r2_score(y_true, y_pred)
        best_param = best_params[region_name]
        best_cv_score = best_scores[region_name]
        
        performance_metrics[region_name] = {
            'MSE': mse_value,
            'R^2': r2_value,
            'Best Parameters': best_param,
            'Best CV Score': best_cv_score
        }
        
        # Plot the residuals
        ax.scatter(y_true, residuals, c='blue', alpha=0.5, s=10)
        ax.axhline(0, color='red', lw=2)
        ax.text(0.05, 0.95, f'MSE: {mse_value:.6f}\nR^2: {r2_value:.2f}\nBest CV: {best_cv_score:.4f}', 
                transform=ax.transAxes, verticalalignment='top')
        ax.set_title(f'{region_name}\n{best_param}')
        ax.set_xlabel('True Values')
        ax.set_ylabel('Residuals')
        
        # Hide axes for subplots that are not used (if num_regions < rows*cols)
        if idx >= num_regions:
            ax.set_visible(False)

    plt.tight_layout()
    plt.show()
    
    return performance_metrics

In [ ]:
gb_performance_metrics = assess_model_performance_all_regions(ds, gradient_boosting_models, best_params, best_scores, scaler_data, predictor_vars)

### Compare Permutation Importance of different models

##### Compare LR test and train

In [ ]:
def compare_permutation_importance(permutation_importances_train, permutation_importances_test, performance_metrics_train, performance_metrics_test, predictor_vars):
    comparison_results = {}

    for region in permutation_importances_train.keys():
        mean_importance_train = pd.Series(permutation_importances_train[region].mean(axis=1), index=predictor_vars)
        mean_importance_test = pd.Series(permutation_importances_test[region].mean(axis=1), index=predictor_vars)

        # Rank the variables, excluding negative importances
        rank_train = mean_importance_train.rank(method='dense', ascending=False).where(mean_importance_train >= 0, np.nan)
        rank_test = mean_importance_test.rank(method='dense', ascending=False).where(mean_importance_test >= 0, np.nan)

        # Special rank for zero importance in both training and test
        zero_importance = (mean_importance_train == 0) & (mean_importance_test == 0)
        max_rank = max(rank_train.max(), rank_test.max()) + 1
        rank_train[zero_importance] = max_rank
        rank_test[zero_importance] = max_rank

        # Calculate agreement
        valid_indices = (mean_importance_train >= 0) | (mean_importance_test >= 0)
        agreement = np.mean(rank_train[valid_indices] == rank_test[valid_indices]) * 100

        # Prepare data for dataframe
        data = {
            'Variable': predictor_vars,
            'Train_Rank': rank_train,
            'Test_Rank': rank_test,
            'Train_Mean_Importance': mean_importance_train,
            'Test_Mean_Importance': mean_importance_test,
            'Train_R2': performance_metrics_train[region]['R2'],
            'Test_R2': performance_metrics_test[region]['R2'],
            'Agreement (%)': agreement
        }

        df = pd.DataFrame(data).sort_values(by='Train_Rank')
        comparison_results[region] = df

    return comparison_results

In [ ]:
def compare_permutation_importance(permutation_importances_train, permutation_importances_test, performance_metrics_train, performance_metrics_test, predictor_vars, zero_threshold=0.001):
    comparison_results = {}

    for region in permutation_importances_train.keys():
        mean_importance_train = pd.Series(permutation_importances_train[region].mean(axis=1), index=predictor_vars)
        mean_importance_test = pd.Series(permutation_importances_test[region].mean(axis=1), index=predictor_vars)

        # Separate zero and non-zero importance variables
        is_zero_train = mean_importance_train.abs() <= zero_threshold
        is_zero_test = mean_importance_test.abs() <= zero_threshold

        # Rank the variables, treating near-zero importance variables separately
        rank_train = mean_importance_train.rank(method='dense', ascending=False, na_option='bottom').astype('Int64')
        rank_test = mean_importance_test.rank(method='dense', ascending=False, na_option='bottom').astype('Int64')

        # Adjust ranks for zero importance variables
        max_rank = max(rank_train.max(), rank_test.max()) + 1
        rank_train[is_zero_train & is_zero_test] = max_rank
        rank_test[is_zero_train & is_zero_test] = max_rank

        # Calculate agreement
        agreement = np.mean(rank_train == rank_test) * 100

        # Prepare data for dataframe
        data = {
            'Variable': predictor_vars,
            'Train_Rank': rank_train,
            'Test_Rank': rank_test,
            'Train_Mean_Importance': mean_importance_train,
            'Test_Mean_Importance': mean_importance_test,
            'Train_R2': performance_metrics_train[region]['R2'],
            'Test_R2': performance_metrics_test[region]['R2'],
            'Agreement (%)': agreement
        }

        df = pd.DataFrame(data).sort_values(by='Train_Rank')
        comparison_results[region] = df

    return comparison_results

In [ ]:
# Example usage
comparison_results = compare_permutation_importance(permutation_importances_train_lr, permutation_importances_test_lr, performance_metrics_train_lr, performance_metrics_test_lr, predictor_vars)

In [ ]:
pd.DataFrame(comparison_results.keys())

In [ ]:
# Displaying the results for one region as an example
region_number = 8
print(list(comparison_results.keys())[region_number])
comparison_results[list(comparison_results.keys())[region_number]]

In [ ]:
# get mean overall agreement and r2

In [ ]:
def compute_agreement_metrics(comparison_results):
    overall_agreement = []
    first_rank_agreement = []

    for region, df in comparison_results.items():
        # Calculate overall agreement for the region
        agreement = df['Agreement (%)'].iloc[0]
        overall_agreement.append(agreement)

        # Check if the top-ranked variable is the same in training and test data
        top_train = df[df['Train_Rank'] == 1.0]['Variable'].iloc[0] if any(df['Train_Rank'] == 1.0) else None
        top_test = df[df['Test_Rank'] == 1.0]['Variable'].iloc[0] if any(df['Test_Rank'] == 1.0) else None
        first_rank_agreement.append(top_train == top_test)

    # Calculate average agreement
    avg_overall_agreement = sum(overall_agreement) / len(overall_agreement)
    avg_first_rank_agreement = sum(first_rank_agreement) / len(first_rank_agreement) * 100

    return avg_overall_agreement, avg_first_rank_agreement

In [ ]:
avg_overall_agreement, avg_first_rank_agreement = compute_agreement_metrics(comparison_results)
print("Average Overall Agreement:", avg_overall_agreement)
print("Average Agreement on First Rank:", avg_first_rank_agreement)

##### Compare xgb test and train

In [ ]:

comparison_results_xgb = compare_permutation_importance(permutation_importances_train_xgb, permutation_importances_test_xgb, performance_metrics_train_xgb, performance_metrics_test_xgb, predictor_vars)

In [ ]:
# Displaying the results for one region as an example
region_number = 15
print(list(comparison_results_xgb.keys())[region_number])
comparison_results_xgb[list(comparison_results_xgb.keys())[region_number]]

In [ ]:
avg_overall_agreement, avg_first_rank_agreement = compute_agreement_metrics(comparison_results_xgb)
print("Average Overall Agreement:", avg_overall_agreement)
print("Average Agreement on First Rank:", avg_first_rank_agreement)

##### Compare xgb and lr

### Build Gaussian Processes model

Probabilistic Outputs: GPs not only provide a prediction for each data point but also give a measure of uncertainty (variance) associated with that prediction. This can help in understanding the confidence of the model in different regions of the input space.

Non-Linear Relationships: GPs, with the right choice of kernel, can model complex non-linear relationships between inputs and outputs, making them more flexible than traditional linear regression models.

Kernel Flexibility: The kernel in a GP defines the relationship between data points. By selecting or designing a kernel that captures the underlying structure of the data, GPs can be adapted to various types of data patterns.

Finally, regarding variable importance: In the context of GPs, interpreting variable importance is not as straightforward as in linear regression. However, one common approach is to examine the sensitivity of the GP's predictions to changes in each input variable. The Automatic Relevance Determination (ARD) kernel, for instance, can adapt its length scale for each dimension of the input space, which can give an indication of the relative importance of each input variable.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.preprocessing import StandardScaler

def prepare_data_for_gp(ds, region_index):
    # Extract data for the given region
    region_data = ds.sel(region=region_index)
    
    # Prepare predictor and target variables
    X = region_data[['pr', 'vpd', 'evspsbl', 'mrro', 'mrso', 'tran', 'lai', 'gpp']].to_array().transpose('lat', 'lon', 'variable')
    y = region_data['bgws'].stack(z=("lat", "lon"))
    
    # Convert X to a 2D array
    X = X.stack(z=("lat", "lon")).transpose('z', 'variable').values
    
    # Create a mask where either X or y has NaN values
    mask = ~np.isnan(y) & ~np.any(np.isnan(X), axis=1)
    
    # Filter out rows using the mask
    X_filtered = X[mask, :]
    y_filtered = y[mask].values  # Convert to numpy array
    
    return X_filtered, y_filtered

def train_gp_for_region(ds, region_name):
    # Prepare data
    X, y = prepare_data_for_gp(ds, region_name)
    
    # Standardize the features
    scaler = StandardScaler().fit(X)
    X_standardized = scaler.transform(X)
    
    # Define the kernel: RBF kernel with ARD + constant term + white noise term for model noise
    # Kernel = Covarianzfunction:
    # Defines relation between input variables
    # 
    kernel = C(1.0, (1e-3, 1e3)) * RBF(np.ones(8), (1e-2, 1e2)) + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))
    
    # Initialize and train GP regressor
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True)
    gp.fit(X_standardized, y)
    
    return gp, scaler

def predict_with_gp(gp, scaler, X):
    # Standardize the features
    X_standardized = scaler.transform(X)
    
    # Predict using the GP regressor
    y_pred, y_std = gp.predict(X_standardized, return_std=True)
    
    return y_pred, y_std

def train_and_predict_for_all_regions(ds):
    region_indices = ds.region.values
    region_names = ds.names.values
    gp_models = {}
    scalers = {}
    predictions = {}
    std_devs = {}
    
    for i, region_index in enumerate(region_indices):
        region_name = region_names[i]
        gp, scaler = train_gp_for_region(ds, region_index)
        X, y = prepare_data_for_gp(ds, region_index)
        
        y_pred, y_std = predict_with_gp(gp, scaler, X)
        
        gp_models[region_name] = gp
        scalers[region_name] = scaler
        predictions[region_name] = y_pred
        std_devs[region_name] = y_std
    
    return gp_models, scalers, predictions, std_devs

In [ ]:
gp_models, scalers, predictions, std_devs = train_and_predict_for_all_regions(ds)

In [ ]:
predictions

In [ ]:
import math
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

def assess_model_performance_all_regions(ds, predictions, std_devs):
    region_names = ds.names.values
    region_indices = ds.region.values
    
    # Determine the number of rows and columns for the subplots based on the number of regions
    num_regions = len(region_names)
    cols = 3  # Assuming you'd like 3 columns
    rows = math.ceil(num_regions / cols)
    
    fig, axs = plt.subplots(rows, cols, figsize=(15, 5*rows))
    
    for i, region_index in enumerate(region_indices):
        region_name = region_names[i]
        
        row = i // cols
        col = i % cols
        
        # Extract true values
        _, y_true = prepare_data_for_gp(ds, region_index)
        
        # Extract predicted values and standard deviations for the region
        y_pred = predictions[region_name]
        y_std = std_devs[region_name]
        
        # Calculate residuals
        residuals = y_pred - y_true
        
        # Calculate the RMSE
        mse_value = mean_squared_error(y_true, y_pred)
        r2_s = r2_score(y_true, y_pred)
        
        # Plot
        ax = axs[row, col]
        ax.scatter(y_true, residuals, c='blue', alpha=0.5, s=10)
        ax.fill_between(y_true, residuals - y_std, residuals + y_std, color='gray', alpha=0.2)
        ax.axhline(0, color='red', lw=2)
        ax.text(0.05, 0.95, f'MSE: {mse_value:.6f}', transform=ax.transAxes, verticalalignment='top')
        ax.text(0.05, 0.9, f'R2 score: {r2_s:.2f}', transform=ax.transAxes, verticalalignment='top')
        ax.set_title(region_name)
        ax.set_xlabel('True Values')
        ax.set_ylabel('Residuals')
        
    # Create legend with explicit handles in the last subplot
    if num_regions < rows * cols:
        last_ax = axs.flatten()[-1]
        last_ax.axis('off')
        legend_elements = [mlines.Line2D([0], [0], color='blue', marker='o', markersize=10, label='Residuals (Observed - Predicted)', linestyle='None'),
                           mlines.Line2D([0], [0], color='gray', alpha=0.2, linewidth=10, label='Prediction Uncertainty (±1 Std. Dev.)'),
                           mlines.Line2D([0], [0], color='red', lw=2, label='Zero Residual Line')]
        last_ax.legend(handles=legend_elements, loc='center')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Example usage:
assess_model_performance_all_regions(ds, predictions, std_devs)

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

def assess_variable_importance_all_regions(ds, gp_models, variable_names):
    region_names = ds.names.values
    region_indices = ds.region.values
    
    # Determine the number of rows and columns for the subplots based on the number of regions
    num_regions = len(region_names)
    cols = 3  # Assuming you'd like 3 columns
    rows = math.ceil(num_regions / cols)
    
    fig, axs = plt.subplots(rows, cols, figsize=(15, 5*rows))
    
    for i, region_index in enumerate(region_indices):
        region_name = region_names[i]
        
        row = i // cols
        col = i % cols
        
        ax = axs[row, col]
        
        # Extract the ARD kernel length scales from the trained GP model for the region
        length_scales = gp_models[region_name].kernel_.k1.k2.length_scale
        
        # Plot
        ax.bar(variable_names, length_scales)
        ax.set_title(region_name)
        ax.set_xlabel('Variable Name')
        ax.set_ylabel('Length Scale')
        ax.tick_params(axis='x', rotation=45)  # Rotate x-axis labels for better visibility
        
    # Remove empty subplots
    for i in range(num_regions, rows*cols):
        fig.delaxes(axs.flatten()[i])
        
    plt.tight_layout()
    plt.show()

In [ ]:
# Call the function
variable_names = ['pr', 'vpd', 'evspsbl', 'mrro', 'mrso', 'tran', 'lai', 'gpp']

assess_variable_importance_all_regions(ds, gp_models, variable_names)

In [ ]:
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import numpy as np
import math

def assess_permutation_importance(ds, gp_models, variable_names, predictions, n_repeats=30):
    """
    Assess the permutation importance of features for all regions.

    Parameters:
    - ds: Dataset containing region names, indices, and bgws values.
    - gp_models: Dictionary containing trained GP models for each region.
    - variable_names: List of variable names.
    - predictions: Predicted values for each region.
    - n_repeats: Number of times to permute a feature.

    Returns:
    - None. Plots the importance.
    """
    
    region_names = ds.names.values
    region_indices = ds.region.values
    y = ds.bgws.values  # Extracting target values from ds
    
    # Determine the number of rows and columns for the subplots based on the number of regions
    num_regions = len(region_names)
    cols = 3  # Assuming you'd like 3 columns
    rows = math.ceil(num_regions / cols)
    
    fig, axs = plt.subplots(rows, cols, figsize=(15, 5*rows))
    
    for region_name in region_names:
        region_index = (ds['region'].values == region_name)

        # Assuming ds has columns for each predictor variable and they are named consistently
        # Extract all predictor variables for this region
        X_region = ds[variable_names][region_index]

        y_region = y[region_index]  
        result = permutation_importance(gp_models[region_name], X_region, y_region, n_repeats=n_repeats)

        # Sort variables by importance
        sorted_idx = result.importances_mean.argsort()
        
        # Plot
        ax.boxplot(result.importances[sorted_idx].T,
                   vert=False, labels=np.array(variable_names)[sorted_idx])
        ax.set_title(region_name)
        ax.set_xlabel('Importance Score')
        
    # Remove empty subplots
    for i in range(num_regions, rows*cols):
        fig.delaxes(axs.flatten()[i])
        
    plt.tight_layout()
    plt.show()

In [ ]:
# Example usage:
assess_permutation_importance(ds, gp_models, variable_names, predictions)

#### Test model performance

In [ ]:
def assess_model_performance_all_regions(ds_change, ds_hist, regression_models, test_data, performance_metrics, predictor_vars, residuals, cv_scores):
    region_names = ds_change.names.values
    region_indices = ds_change.region.values
    
    num_regions = len(region_names)
    plots_per_region = 3  # Number of plots per region
    total_plots = num_regions * plots_per_region
    cols = 3  # Number of columns (one for each type of plot)
    rows = math.ceil(total_plots / cols)  # Calculate the total number of rows needed

    fig, axs = plt.subplots(rows, cols, figsize=(20, rows * 5), squeeze=False)

    plot_idx = 0  # Initialize plot index
    for region in region_indices:
        region_name = ds_change.names.sel(region=region).values.item()
        X_test, y_test = test_data[region_name]
        model = regression_models[region_name]
        y_pred = model.predict(X_test)
        resids = residuals[region_name]

        mse_value = performance_metrics[region_name]['MSE']
        r2_value = performance_metrics[region_name]['R2']
        cv_score = np.mean(cv_scores[region_name])  # Average CV score

        # Plot 1: Residuals
        ax_resid = axs[plot_idx // cols, plot_idx % cols]
        ax_resid.scatter(y_test, resids, c='blue', alpha=0.5, s=10, label='Residuals')
        ax_resid.axhline(0, color='red', lw=2, label='Zero Residual')
        ax_resid.set_xlabel('True Values')
        ax_resid.set_ylabel('Residuals')
        ax_resid.set_title(f"{region_name} - Residuals")
        ax_resid.legend()
        plot_idx += 1

        # Plot 2: Predictions vs True Values
        ax_pred = axs[plot_idx // cols, plot_idx % cols]
        ax_pred.scatter(y_test, y_pred, c='green', alpha=0.5, s=10, label='Predictions')
        ax_pred.plot(y_test, y_test, color='orange', label='Ideal Prediction')
        ax_pred.set_xlabel('True Values')
        ax_pred.set_ylabel('Predicted Values')
        ax_pred.set_title(f"{region_name} - Predictions")
        ax_pred.legend()
        plot_idx += 1

        # Plot 3: Density Plot
        ax_density = axs[plot_idx // cols, plot_idx % cols]
        sns.kdeplot(y_test, ax=ax_density, label='Actual Values', fill=True)
        sns.kdeplot(y_pred, ax=ax_density, label='Predicted Values', fill=True)
        ax_density.set_xlabel('Values')
        ax_density.set_title(f"{region_name} - Density Plot")
        ax_density.legend()
        plot_idx += 1

        # Include performance metrics in the text or title
        ax_density.text(0.05, 0.95, f'MSE: {mse_value:.6f}\nR^2: {r2_value:.2f}\nCV: {cv_score:.2f}', transform=ax_density.transAxes, verticalalignment='top')

    plt.tight_layout()
    plt.show()

In [ ]:
# relative change scaled
assess_model_performance_all_regions(ds, ds_hist_ensemble_metric, regression_models, test_data, performance_metrics, predictor_vars, residuals, cv_scores)


In [ ]:
assess_model_performance_all_regions(ds, ds_hist_ensemble_metric, regression_models, test_data, performance_metrics, predictor_vars, residuals, cv_scores)


In [ ]:
assess_model_performance_all_regions(ds, ds_hist_ensemble_metric, regression_models, test_data, performance_metrics, predictor_vars, residuals, cv_scores)


#### Test optimal variable subset

In [ ]:
from itertools import chain, combinations

In [ ]:
all_combinations = list(all_subsets(predictor_vars)) 
print(f"Number of variable combinations with at least 4 variables: {len(all_combinations)}")

In [ ]:
def all_subsets(ss, min_length=4):
    """Generate all combinations of the elements in `ss` with a minimum length of `min_length`."""
    return chain(*map(lambda x: combinations(ss, x), range(min_length, len(ss)+1)))

In [ ]:
def test_variable_combinations(ds, all_vars, predictant):
    # Get all possible combinations of predictor variables with at least 4 variables
    all_combinations = list(all_subsets(all_vars))
    print(f"Number of variable combinations: {len(all_combinations)}")

    # Prepare a list to store the performance metrics
    performance_list = []

    for combination in all_combinations:
        # Convert the tuple to a list of variables for this combination
        current_vars = list(combination)

        # Train the models using the current combination of variables
        _, regression_models, _, _, _, _ = train_multivariate_models(ds, current_vars, predictant)
       
        # Assess model performance for all regions
        for region in ds.region.values:
            region_name = ds.names.sel(region=region).values.item()
            
            # Prepare the data for the region
            df = ds.sel(region=region).to_dataframe().dropna()
            X = df[current_vars]
            y_true = df[predictant].values

            # Standardize the predictor variables
            scaler = StandardScaler()
            X_standardized = scaler.fit_transform(X) # standarized using this method might be wrong cause we need the direction. Standarize 
            #with mean 
            
            # Predict using the trained model
            model = regression_models[region_name]
            y_pred = model.predict(X_standardized)
            
            # Calculate performance metrics
            mse = mean_squared_error(y_true, y_pred)
            r2 = r2_score(y_true, y_pred)
            
            # Append the performance metrics for this region and variable combination to the list
            performance_list.append({
                'Variables': ', '.join(current_vars),
                'MSE': mse,
                'R^2': r2
            })

    # Convert the list to a DataFrame
    performance_summary = pd.DataFrame(performance_list)
    
    # Calculate the aggregated performance for each variable combination across all regions
    aggregated_performance = performance_summary.groupby('Variables').agg(['mean', 'min', 'max']).reset_index()
    aggregated_performance.columns = [' '.join(col).strip() for col in aggregated_performance.columns.values]

    # Sort the results by Mean MSE and Mean R^2
    aggregated_performance.sort_values(by=['MSE mean', 'R^2 mean'], ascending=[True, False], inplace=True)

    # Select and rename the columns to only include the required statistics
    final_table = aggregated_performance[['Variables', 'MSE mean', 'MSE min', 'MSE max', 'R^2 mean', 'R^2 min', 'R^2 max']]
    final_table.rename(columns={
        'MSE mean': 'Mean MSE',
        'MSE min': 'Min MSE',
        'MSE max': 'Max MSE',
        'R^2 mean': 'Mean R^2',
        'R^2 min': 'Min R^2',
        'R^2 max': 'Max R^2'
    }, inplace=True)

    return final_table

In [ ]:
# Example usage:
pd.set_option('display.max_rows', None)  
test_variable_combinations(ds, predictor_vars, 'bgws')

#### Cluster regions based on regression coefficients 

In [ ]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist

def cluster_regions(coefficients_df, max_clusters=40):
    """
    This function finds the optimal number of clusters using the elbow method
    and clusters regions based on their coefficients.
    
    :param coefficients_df: A pandas DataFrame containing the coefficients with regions as the index.
    :param max_clusters: The maximum number of clusters to test for the elbow method.
    :return: A tuple of pandas DataFrames with the first containing the coefficients and an additional
             column 'Cluster', and the second containing the centroids of each cluster.
    """
    
    # Find the optimal number of clusters using the elbow method
    sum_of_squared_distances = []
    K = range(1, max_clusters + 1)
    for k in K:
        km = KMeans(n_clusters=k,  n_init=10, random_state=42)
        km = km.fit(coefficients_df)
        sum_of_squared_distances.append(km.inertia_)

    # Plot the elbow curve
    plt.plot(K, sum_of_squared_distances, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Sum of squared distances')
    plt.title('Elbow Method For Optimal k')
    plt.show()

    # Ask user for the optimal number of clusters
    n_clusters = int(input("Enter the optimal number of clusters: "))
    
    # Use the KMeans algorithm to find clusters with the optimal number of clusters
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    kmeans.fit(coefficients_df)

    # Assign the clusters to each region
    clusters = kmeans.labels_
    centroids = kmeans.cluster_centers_

    # Calculate the distance of each region's coefficients to the centroid of its cluster
    distances_to_centroid = cdist(coefficients_df, centroids, 'euclidean')
    min_distances = distances_to_centroid.min(axis=1)

    # Add the cluster labels and distances to the DataFrame
    df = coefficients_df.copy()
    df['Cluster'] = clusters
    df['Distance_to_Centroid'] = (min_distances * 100).round(decimals=2)
    df.index.name = 'Region'
    df.reset_index(inplace=True)
    
    # Order the dataframe by cluster label and distance to centroid
    clustered_df = df.sort_values(['Cluster', 'Distance_to_Centroid'])

    # Get the centroids
    centroids_df = pd.DataFrame(centroids, columns=coefficients_df.columns)
    centroids_df['Cluster'] = range(n_clusters)

    return clustered_df, centroids_df

In [ ]:
from sklearn.cluster import KMeans

def cluster_regions(coefficients_df, max_clusters=40):
    """
    This function finds the optimal number of clusters using the elbow method
    and clusters regions based on their coefficients.

    :param coefficients_df: A pandas DataFrame containing the coefficients with regions as the index.
    :param max_clusters: The maximum number of clusters to test for the elbow method.
    :return: A pandas DataFrame with an additional column 'Cluster' indicating the cluster for each region.
    """
    
    # Find the optimal number of clusters using the elbow method
    sum_of_squared_distances = []
    K = range(1, max_clusters + 1)
    for k in K:
        km = KMeans(n_clusters=k,  n_init=10, random_state=42)
        km = km.fit(coefficients_df)
        sum_of_squared_distances.append(km.inertia_)

    # Plot the elbow curve
    plt.plot(K, sum_of_squared_distances, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Sum of squared distances')
    plt.title('Elbow Method For Optimal k')
    plt.show()

    # Ask user for the optimal number of clusters
    n_clusters = int(input("Enter the optimal number of clusters: "))
    
    # Use the KMeans algorithm to find clusters with the optimal number of clusters
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    kmeans.fit(coefficients_df)

    # Assign the clusters to each region
    clusters = kmeans.labels_

    # Add the cluster labels to the DataFrame
    df = coefficients_df.copy()
    df['Cluster'] = clusters
    df.index.name = 'Region'
    df.reset_index(inplace=True)
    
    # Order the dataframe by cluster label
    clustered_df = df.sort_values('Cluster')

    return clustered_df

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial import distance_matrix

def custom_distance(u, v):
    # Custom distance metric:
    # If any signs differ, the distance is increased
    sign_diff = np.sign(u) != np.sign(v)
    if sign_diff.any():  # If any signs differ, apply the penalty
        return np.linalg.norm(u - v) * 1
    else:
        return np.linalg.norm(u - v)
    
def cluster_regions(coefficients_df, n_clusters=12):
    # Create a new DataFrame for clustering to avoid modifying the original data
    X = coefficients_df.copy()

    # Compute the custom distance matrix
    dist_matrix = distance_matrix(X.values, X.values, p=2)
    for i in range(dist_matrix.shape[0]):
        for j in range(dist_matrix.shape[1]):
            if i != j:  # No need to penalize the diagonal
                dist_matrix[i, j] = custom_distance(X.values[i], X.values[j])

    # Perform Agglomerative Clustering with the precomputed distance matrix
    clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity='precomputed', linkage='complete')
    clusters = clustering.fit_predict(dist_matrix)

    # Assign the clusters to each region
    coefficients_df['Cluster'] = clusters
    
    # Order the dataframe by cluster label
    clustered_df = coefficients_df.sort_values('Cluster')
    clustered_df.index.name = 'Region'
    clustered_df.reset_index(inplace=True)
    
    return clustered_df

In [ ]:
clustered_df, centroids_df = cluster_regions(coefficients_df)
clustered_df

#### Plot clusters on map

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

colors = [(34/255, 139/255, 34/255), (1, 1, 1), (60/255, 145/255, 230/255)]  # Green -> White -> Blue
n_bins = [3]  # Discretizes the interpolation into bins
cmap_name = 'custom_div_cmap'

cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)

# To test and display the colormap
fig, ax = plt.subplots(figsize=(6, 1))
ax.set_title('Custom Diverging Colormap')
plt.imshow(np.linspace(0, 1, 256).reshape(1, -1), aspect='auto', cmap=cm)
plt.axis('off')
plt.show()

In [ ]:
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
import regionmask
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

def split_polygon(polygon, meridian=180):
    """
    Splits a Shapely polygon into two polygons at a specified meridian
    """
    minx, miny, maxx, maxy = polygon.bounds
    if maxx > meridian and minx < -meridian:
        # Polygon crosses the antimeridian
        left_poly = []
        right_poly = []
        for x, y in polygon.exterior.coords:
            if x >= meridian:
                right_poly.append((x - 360, y))  # Wraparound for the right side
            else:
                left_poly.append((x, y))
        return [Polygon(left_poly), Polygon(right_poly)]
    else:
        return [polygon]  # Wrap the single polygon in a list for consistency

In [ ]:
def plot_clusters_and_regions(ds_ensmed_glob, ds, clustered_df):
    # Initialize the plot with a cartopy projection
    fig = plt.figure(figsize=(30, 15))
    ax_main = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    
    # Plot the 'bgws' variable from the dataset
    ds_ensmed_glob['bgws'].plot(ax=ax_main, vmin=-0.2, vmax=0.2, cmap=cm, transform=ccrs.PlateCarree(), add_colorbar=False)
    
    # Add coastlines and gridlines
    ax_main.coastlines()
    ax_main.tick_params(axis='both', which='major', labelsize=20)
    gridlines = ax_main.gridlines(draw_labels=True, color='black', alpha=0.2, linestyle='--')
    gridlines.top_labels = gridlines.right_labels = False
    gridlines.xlabel_style = {'size': 18}
    gridlines.ylabel_style = {'size': 18}
    
    # Get region bounds using regionmask
    land_regions = regionmask.defined_regions.ar6.land
    
    # Create a mapping from region names to region numbers
    region_name_to_number = dict(zip(ds.names.values, ds.region.values))
    
    # Map the region names to the same order as ds.names.values
    region_to_cluster_map = dict(zip(clustered_df['Region'], clustered_df['Cluster']))

    # Now create a new column in ds that maps the region names to cluster numbers
    # This assumes ds.names.values has the same region names as in clustered_df['Region']
    ds['Cluster'] = [region_to_cluster_map[name] for name in ds.names.values]

    # Convert the 'Cluster' DataArray to a NumPy array and get unique values
    unique_clusters = np.unique(ds['Cluster'].values)

    # Prepare colors for clusters - this assumes a finite number of clusters
    cluster_colors = plt.cm.tab20b(np.linspace(0, 1, len(unique_clusters)))


    # Loop over the regions and plot the cluster numbers with abbreviations
    for region_name, cluster_number in zip(ds.names.values, ds['Cluster']):
        reg_num = region_name_to_number[region_name]
        region_polygons = land_regions[reg_num].polygon
        
        region_abbr = ds.abbrevs.values[ds.region.values == reg_num][0]  # Assuming this gives us the correct abbreviation
        cluster_color = cluster_colors[cluster_number]  # Get the color for the cluster
        
        # Fetch the polygon or polygons for this region
        region_obj = land_regions[reg_num]
        if hasattr(region_obj, 'polygons'):
            # If the attribute is 'polygons', we assume it's iterable (e.g., a list of Polygon objects)
            region_polygons = region_obj.polygons
        elif hasattr(region_obj, 'polygon'):
            # If there's only one Polygon, we wrap it in a list to make it iterable
            region_polygons = [region_obj.polygon]
        else:
            raise AttributeError(f"The region object does not have 'polygons' or 'polygon' attribute.")

        for region_polygon in region_polygons:
            # If the polygon crosses the antimeridian, split it
            split_polys = split_polygon(region_polygon)

            # Handle both Polygon and MultiPolygon types after splitting
            for poly in split_polys:
                if isinstance(poly, Polygon):
                    features_to_plot = [poly]
                elif isinstance(poly, MultiPolygon):
                    features_to_plot = list(poly.geoms)
                else:
                    raise TypeError(f"Unhandled geometry type: {type(poly)}")

                for feature_poly in features_to_plot:
                    feature = ShapelyFeature([feature_poly], ccrs.PlateCarree(), edgecolor=cluster_color, facecolor='none', linewidth=2)
                    ax_main.add_feature(feature)

            # Calculate the centroid for text placement using features_to_plot
            centroids = [feature_poly.centroid for feature_poly in features_to_plot]
    
            text_lon, text_lat = max(centroids, key=lambda c: c.x).coords[0]  # Use the easternmost centroid
        
            # Ensure cluster_number is a plain integer if it's a single-item array or DataArray
            if isinstance(cluster_number, np.ndarray) and cluster_number.size == 1:
                cluster_number = cluster_number.item()  # Converts a one-element array to a scalar
            elif isinstance(cluster_number, xr.DataArray) and cluster_number.ndim == 0:
                cluster_number = cluster_number.values.item()  # Gets the scalar value from a 0-dim DataArray

            # Annotate the cluster number for each region
            ax_main.text(text_lon, text_lat, f"{region_abbr}\n{cluster_number}",
                         horizontalalignment='center', verticalalignment='center', transform=ccrs.PlateCarree(),
                         fontsize=20, bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))

    plt.show()

In [ ]:
plot_clusters_and_regions(ds_ensmed_glob, ds, clustered_df)

In [ ]:
clustered_df

In [ ]:
def plot_simple_map(ds_ensmed_glob, projection, save_fig=False):
    fig = plt.figure(figsize=(30, 15))
    ax_main = fig.add_subplot(1, 1, 1, projection=projection)
    
    # Plot the 'bgws' variable from the dataset
    img = ds_ensmed_glob['bgws'].plot(ax=ax_main, vmin=-0.6, vmax=0.6,  cmap=cm, transform=ccrs.PlateCarree(), add_colorbar=False) 
    
    # Add coastlines and gridlines
    ax_main.coastlines()
    ax_main.tick_params(axis='both', which='major', labelsize=20)
    gridlines = ax_main.gridlines(draw_labels=True, color='black', alpha=0.2, linestyle='--')
    gridlines.top_labels = gridlines.right_labels = False
    gridlines.xlabel_style = {'size': 18}
    gridlines.ylabel_style = {'size': 18}
    
    # Colorbar
    cbar_ax = fig.add_axes([0.314, 0.1, 0.4, 0.025]) #left, bottom, width, height
    cbar = fig.colorbar(img, cax=cbar_ax, extend='both', orientation='horizontal')
    cbar.set_label("Blue-Green Water Share", fontsize=22, weight='bold', labelpad=15) 
    cbar.ax.tick_params(labelsize=18)
   
    plt.show()
    
    # Safe figure
    if save_fig:
        savepath = os.path.join('../..', 'results', 'CMIP6', 'historical', 'time', 'median', 'bgws')
        os.makedirs(savepath, exist_ok=True)
        filename = f'Ensemble_median.1985-2014.bgws.historical.pdf'
        filepath = os.path.join(savepath, filename)
        fig.savefig(filepath, dpi=600, bbox_inches='tight', format='pdf')
    else:
        filepath = 'Figure not saved. If you want to save the figure add save_fig=True to the function call'
        
    return filepath

In [ ]:
plot_simple_map(ds_ensmed_glob, ccrs.Robinson(), save_fig=True) # Robinson PlateCarree

In [ ]:
plot_simple_map(ds_ensmed_glob, ccrs.PlateCarree(), save_fig=False)